In [ ]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_predict

In [ ]:
df_train = pd.read_csv('tmlcc-2021/train.csv')
df_test = pd.read_csv('tmlcc-2021/test.csv')
df_pre_train = pd.read_csv('tmlcc-2021/pretest.csv')

In [ ]:
df_train.head()

In [ ]:
df_train.info()

In [ ]:
df_train = df_train.dropna()

In [ ]:
df_train = df_train.loc[(df_train['void_volume [cm^3/g]'] > 0) & (df_train['void_fraction'] > 0) & (df_train['surface_area [m^2/g]'] > 0) & (df_train['volume [A^3]'] > 0)]

In [ ]:
df_train, y = df_train.loc[:, 'volume [A^3]':'heat_adsorption_CO2_P0.15bar_T298K [kcal/mol]'], df_train.loc[:,'CO2_working_capacity [mL/g]']

In [ ]:
dummies_meatl = pd.get_dummies(df_train['metal_linker'], prefix='met')
dummies_topology = pd.get_dummies(df_train['metal_linker'], prefix='topo')
df_onehot = pd.concat([dummies_meatl, dummies_topology], axis = 1)

In [ ]:
df_train = df_train.drop(['functional_groups', 'metal_linker', 'organic_linker1', 'organic_linker2', 'topology', 'heat_adsorption_CO2_P0.15bar_T298K [kcal/mol]'], axis=1)

In [ ]:
df_train.head()

In [ ]:
df_train

In [ ]:
normalized_df = (df_train-df_train.mean()) / df_train.std()

In [ ]:
normalized_df

In [ ]:
df_train = pd.concat([normalized_df, df_onehot], axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_train, y, test_size=0.33, random_state=42)

In [ ]:
reg = LinearRegression()

In [ ]:
reg.fit(X_train, y_train)

In [ ]:
y_pred = reg.predict(X_test)

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
np.log(mean_absolute_error(y_test, y_pred))

In [ ]:
df_test

### Test

In [ ]:
df_test = df_test.loc[:, 'volume [A^3]':'heat_adsorption_CO2_P0.15bar_T298K [kcal/mol]']

dummies_meatl = pd.get_dummies(df_test['metal_linker'], prefix='met')
dummies_topology = pd.get_dummies(df_test['metal_linker'], prefix='topo')
df_onehot = pd.concat([dummies_meatl, dummies_topology], axis = 1)

df_test = df_test.drop(['functional_groups', 'metal_linker', 'organic_linker1', 'organic_linker2', 'topology', 'heat_adsorption_CO2_P0.15bar_T298K [kcal/mol]'], axis=1)

In [ ]:
normalized_df = (df_test-df_test.mean()) / df_test.std()

In [ ]:
df_test = pd.concat([normalized_df, df_onehot], axis=1)

In [ ]:
pred = reg.predict(df_test)

In [ ]:
ID = np.arange(68614, 85614)

In [ ]:
pd.DataFrame({'id':ID, 'CO2_working_capacity [mL/g]': pred}).set_index('id').to_csv('wonderland01.csv')